In [ ]:
 import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/AD Identification using SATD'

Mounted at /content/drive
/content/drive/My Drive/AD Identification using SATD


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline

import pandas as pd


#Read the dataset


In [ ]:
import pandas as pd
liu_ = '/content/drive/My Drive/AD Identification using SATD/liu_datset_processed.csv'
liu_ = pd.read_csv(liu_, low_memory=False)

In [ ]:
liu_['Comments'].fillna('', inplace=True)
liu_['TDType'] = liu_['TDType'].astype(str)

values_to_remove = ['MULTITHREAD', 'nan', 'removeType']
replacement_value = 'WITHOUT_CLASSIFICATION'
liu_['TDType'].replace(values_to_remove, replacement_value, inplace=True)

liu_['Comments'] = liu_['Comments'].str.replace('content=', '', regex=False)
liu_['Comments'] = liu_['Comments'].str.replace('"', '', regex=False)

liu_ = liu_.drop_duplicates(subset=['Comments', 'TDType'])
liu_['TDType'] = liu_['TDType'].replace('removeType', 'WITHOUT_CLASSIFICATION')

In [ ]:
# Count the number of duplicate rows in the DataFrame
num_duplicates = liu_.duplicated().sum()

print(f"Number of duplicate rows: {num_duplicates}")


Number of duplicate rows: 0


##Tfidf

In [ ]:
data = liu_

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(data['Comments'], data['TDType'], test_size=0.2, random_state=42)
# Further split the training set into training and validation sets
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('TfidfVectorizer', TfidfVectorizer()),
    ('scaler', StandardScaler(with_mean=False)),  # Scale feature vectors
    ('clf', SVC(random_state=42, kernel='rbf', class_weight='balanced', probability=True))  # SVM
])


param_grid = {
    'clf__C': [0.01, 1],  # Regularization strength


}

grid_search = GridSearchCV(estimator=pipeline,
                           param_grid=param_grid,
                           cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                           scoring='accuracy',
                           n_jobs=-1,
                           verbose=2)

# Fit the grid search to the training data
grid_search.fit(X_train_final, y_train_final)


print("Best parameters found:", grid_search.best_params_)

# Evaluate the best model on the validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)
conf_matrix_val = confusion_matrix(y_val, y_val_pred)
classification_rep_val = classification_report(y_val, y_val_pred)

print("\nValidation Confusion Matrix:")
print(conf_matrix_val)
print("\nValidation Classification Report:")
print(classification_rep_val)

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classification_rep_test = classification_report(y_test, y_test_pred)

print("\nTest Confusion Matrix:")
print(conf_matrix_test)
print("\nTest Classification Report:")
print(classification_rep_test)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best parameters found: {'clf__C': 1}

Validation Confusion Matrix:
[[  36    0    2   80    0    2    0   23]
 [   0   18    2   37    0    2    1    7]
 [   2    2   37   45    0    2    3    8]
 [  21   12   17 1423    4   73    8  158]
 [   0    0    0    6   10    0    0    4]
 [   1    2    3  134    3  173    6   26]
 [   1    0    1   48    0    3   48    4]
 [  14    6   10  914    0   14   28 2736]]

Validation Classification Report:
                        precision    recall  f1-score   support

             ALGORITHM       0.48      0.25      0.33       143
         COMPATIBILITY       0.45      0.27      0.34        67
                DEFECT       0.51      0.37      0.43        99
                DESIGN       0.53      0.83      0.65      1716
         DOCUMENTATION       0.59      0.50      0.54        20
        IMPLEMENTATION       0.64      0.50      0.56       348
                  TEST       0.51      0.46 